In [3]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.data.path.append('.')
import polars as pl

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aydin.firdouzov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aydin.firdouzov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:


df = pl.read_parquet('hf://datasets/LocalDoc/news_azerbaijan/data/train-*.parquet')
df = df.to_pandas()
df.head()

OSError: object-store error: Generic HTTP error: Request error: Error performing GET https://huggingface.co/datasets/LocalDoc%2Fnews_azerbaijan/resolve/main/data%2Ftrain-00001-of-00002.parquet in 89.7272ms - Server returned non-2xx status code: 429 Too Many Requests: <!DOCTYPE html>
<html class="" lang="en">
<head>
    <meta charset="utf-8" />
    <meta
            name="viewport"
            content="width=device-width, initial-scale=1.0, user-scalable=no"
    />
    <meta
            name="description"
            content="We're on a journey to advance and democratize artificial intelligence through open source and open science."
    />
    <meta property="fb:app_id" content="1321688464574422" />
    <meta name="twitter:card" content="summary_large_image" />
    <meta name="twitter:site" content="@huggingface" />
    <meta
            property="og:title"
            content="Hugging Face - The AI community building the future."
    />
    <meta property="og:type" content="website" />

In [ ]:
dataset = df['text'].values
dataset[:3]

array(['Gürcüstan prezidenti Mixail Saakaşvili Rusiyaya qarşı daha bir kəskin ittiham səsləndirib. Gürcüstanın "Apsyn.ge" saytının verdiyi məlumata görə, Saakaşvili bildirib ki, Bakı-Supsa nef kəməri vasitəsi ilə Rusiya Gürcüstan torpaqlarını ələ keçirmək istəyir: "Artıq xeyli torpaq sahəsi ələ keçirilib. Onların məqsədi Bakı-Supsa strateji neft kəmərini də tikanlı məftillərlə əhatə etməkdir. Hamımız özümüzü ələ almalıyıq. İndi bölücülük, kin, xal qazanmaq vaxtı deyil. Gürcüstanın məqsədi torpaqlarımızın təhlükəli ilhaqını birdəfəlik dayandırmaqdır". "Axar.az" saytı olaraq bu barədə vaxtilə Rusiyada səfir kimi fəaliyyət göstərmiş politoloq Hikmət Hacızadədən münasibət öyrəndik. Hikmət bəy məsələni belə bir lakonik fikirlə ifadə etdi: "Rusiya yenə öz ampluasındadır. Demək istəyir ki, harada biz varıq, ora bizimkidir. Harada neft varsa, ora da bizimdir. Təxminən belə bir lotuluq etmək istəyir".',
       '"Axar.az" saytının İsrail mətbuatına istinadən verdiyi məlumata görə, İsrailin Baş N

In [ ]:
def lower(text: list[str]) -> list[str]:
  """
  Lowerize and tokenize the documents

  :param text: list of documents
  """
  lower_text = text.lower()
  lower_text = nltk.word_tokenize(lower_text)
  return lower_text

In [ ]:
dataset = dataset[dataset != None]
train_data = []
for i,text in enumerate(dataset):
  train_data.append(lower(text))

In [ ]:
def word_count(data: list[str]) -> dict:
  """
  Word counter by documents

  :param data: tokenized document dataset
  """
  word_counts = {}
  for text in data:
    for word in text:
      if word not in word_counts:
        word_counts[word] = 1
      else:
        word_counts[word] += 1
  return word_counts

In [ ]:
train_data_count = word_count(train_data)

In [ ]:
def threshold_count(count: dict,threshold : int) -> dict:
  """
  Filter the counter dictionary with threshold
  
  :param count: word counter dictionary
  :param threshold: threshold value for filtering the dictionary
  """
  word_count = {}
  for word,count in count.items():
    if count>threshold:
      word_count[word] = count
  return word_count

In [ ]:
train_data_threshold_count = threshold_count(train_data_count,300)

In [ ]:
def unk_words(test_data: dict,vocab: list[str],unk_token='<UNK>') -> dict:  
  """
  Assign Unkown tag to words which cann't pass the threshold filter

  :param test_data: Test dataset for applying tag
  :param vocab: Vocabulary of the data
  :param unk_token: String variable of unknown tag
  """
  for text in test_data:
    for i,word in enumerate(text):
      if word not in vocab:
        text[i] = unk_token
  return test_data

In [ ]:
train_data_unk = unk_words(train_data,train_data_threshol_count.keys())

NameError: name 'train_data' is not defined

In [ ]:
def define_ngrams(data: dict,n: int,start_token='<s>':str,end_token='</s>':str) -> dict:
  """
  Function helps to create n grams

  :param data: unknown tagged dataset
  :param n: n-gram's n
  :param start_token: start token for each document
  :param end_token: end token for each document 
  """
  ngram_count = {}
  for text in data:
    text = [start_token]*n + text + [end_token]
    for word in range(len(text)-n+1):
      ngram = tuple(text[word:word+n])
      ngram_count[ngram] = ngram_count.get(ngram,0)+1
  return ngram_count

In [ ]:
train_1gram_count = define_ngrams(train_data_unk,1)
train_2gram_count = define_ngrams(train_data_unk,2)
train_3gram_count = define_ngrams(train_data_unk,3)

In [30]:
def probability(word,prev_word,train_nminus1gram_count,train_ngram_count,k=1.0):
  previous_word_count = train_nminus1gram_count[(prev_word,)] if (prev_word,) in train_nminus1gram_count else 0
  n_gram = tuple([prev_word]) + (word,)
  n_gram_count = train_ngram_count[n_gram] if n_gram in train_ngram_count else 0
  numerator = n_gram_count + k
  denomiator = previous_word_count + k*len(train_nminus1gram_count)
  return numerator/denomiator

In [31]:
probability('jurnalist','hərbi',train_1gram_count,train_2gram_count,5)

0.00044413937901239555

In [32]:
def all_probabilities(previous_word, nminus1gram_count, ngram_count, start_token, end_token):
    probabilities = {}
    for word in nminus1gram_count.keys():
        word_str = word[0]  # Extract the string from the tuple
        proba = probability(word_str, previous_word, nminus1gram_count, ngram_count, 5)
        probabilities[word] = proba
    return probabilities

In [33]:
data_proba = all_probabilities('hərbi',train_2gram_count,train_3gram_count,'<s>','</s>')

In [34]:
data_proba = dict(sorted(data_proba.items(), key=lambda item: item[1], reverse=True))

In [ ]:
sorted_items = sorted(data_proba.items(), key=lambda item: item[1], reverse=True)

for key, value in sorted_items:
    if key != ('<UNK>',):
        top_item = {key: value}
        break

print(top_item.keys())

{('<s>', '<s>'): 1.94604436319493e-07,
 ('<s>', 'gürcüstan'): 1.94604436319493e-07,
 ('gürcüstan', 'prezidenti'): 1.94604436319493e-07,
 ('prezidenti', 'mixail'): 1.94604436319493e-07,
 ('mixail', 'saakaşvili'): 1.94604436319493e-07,
 ('saakaşvili', 'rusiyaya'): 1.94604436319493e-07,
 ('rusiyaya', 'qarşı'): 1.94604436319493e-07,
 ('qarşı', 'daha'): 1.94604436319493e-07,
 ('daha', 'bir'): 1.94604436319493e-07,
 ('bir', 'kəskin'): 1.94604436319493e-07,
 ('kəskin', 'ittiham'): 1.94604436319493e-07,
 ('ittiham', 'səsləndirib'): 1.94604436319493e-07,
 ('səsləndirib', '.'): 1.94604436319493e-07,
 ('.', 'gürcüstanın'): 1.94604436319493e-07,
 ('gürcüstanın', '``'): 1.94604436319493e-07,
 ('``', '<UNK>'): 1.94604436319493e-07,
 ('<UNK>', "''"): 1.94604436319493e-07,
 ("''", 'saytının'): 1.94604436319493e-07,
 ('saytının', 'verdiyi'): 1.94604436319493e-07,
 ('verdiyi', 'məlumata'): 1.94604436319493e-07,
 ('məlumata', 'görə'): 1.94604436319493e-07,
 ('görə', ','): 1.94604436319493e-07,
 (',', 'sa

In [35]:
sorted_items = sorted(data_proba.items(), key=lambda item: item[1], reverse=True)

for key, value in sorted_items:
    if key != ('<UNK>',):
        top_item = {key: value}
        break

print(top_item.keys())

dict_keys([('<s>', '<s>')])
